# Run Train of Bubble-Agent (DQN)

- Team: TToBoT
- Member: { Sejun, Steve, Victor } @kaist

## Objective

- run training simultaneously w/ notebook
- to compare the performance of traing 


## For Competition

1. prepare the final trained IQN Model (checkpoint w/ 100 iteration)
2. need to customize of env.step()
   - it should work only 1 live. (later, we can use 3 lives)
   - need to enumerate all stage (1~99) level w/ at least 10 (250,000 x 10) (8hr x 100 = 800/33d) iteration. (model should be same)
   - using origin step(). do train w/ random level, do loop (iteration) forever! (final training)
3. in final competion, it will load the latest checkpoint for initial Model paramters.
4. win the competition!!


In [1]:
import os, sys, gin

# use parent folder as shared lib path..
if "../" not in sys.path:
    sys.path.append("../")

# major libraries
import gin.tf
import seaborn as sns
import matplotlib.pyplot as plt
from absl import flags
import numpy as np
import tensorflow as tf

# show tf version.
print('! tf.ver = {}'.format(tf.__version__))

! tf.ver = 1.15.2


In [2]:
# Globals
# BASE_PATH = './!experimental_results_bubble/run3'

# let Dopamine .py files to be imported as modules in Jupiter notebook
module_path = os.path.abspath(os.path.join('../dopamine'))
if module_path not in sys.path:
    sys.path.append(module_path)
    print(module_path)

/tf/dopamine


In [3]:
# try to load `Dopamine` libraries
import bubble
from dopamine.colab import utils as colab_utils

## Train Bubble w/ DQN

In [4]:
# @title Load the configuration for DQN.

# DQN_PATH = os.path.join(BASE_PATH, 'rainbow')
# Modified from dopamine/agents/dqn/config/dqn_cartpole.gin

# CONFIG FOR DQN (see @bubble/dqn_nature.gin)
gin_config = '''
# run_experiment
# python -um dopamine.discrete_domains.train --base_dir=/tmp/bubble --gin_files='bubble/dqn_nature.gin'
# python -um dopamine.discrete_domains.train --base_dir=/tmp/bubble --gin_files='bubble/dqn_nature.gin' --gin_bindings='DQNAgent.tf_device="/cpu:*"'
 

# Hyperparameters used in Mnih et al. (2015).
import dopamine.discrete_domains.atari_lib
import dopamine.discrete_domains.run_experiment
import dopamine.agents.dqn.dqn_agent
import dopamine.replay_memory.circular_replay_buffer
import gin.tf.external_configurables

import bubble.retro_lib
import bubble.bubble_agent

retro_lib.create_retro_environment.game_name = 'BubbleBobble'
retro_lib.create_retro_environment.level = 1
Runner.create_environment_fn = @retro_lib.create_retro_environment
create_agent.agent_name = 'dqn'
RetroPreprocessing.wall_offset = 0

DQNAgent.gamma = 0.99
DQNAgent.update_horizon = 1
DQNAgent.min_replay_history = 50000  # agent steps
DQNAgent.update_period = 4
DQNAgent.target_update_period = 10000  # agent steps
DQNAgent.epsilon_train = 0.1
DQNAgent.epsilon_eval = 0.05
DQNAgent.epsilon_decay_period = 1000000  # agent steps
DQNAgent.tf_device = '/gpu:1'  # use '/cpu:*' for non-GPU version
DQNAgent.optimizer = @tf.train.RMSPropOptimizer()

tf.train.RMSPropOptimizer.learning_rate = 0.00025
tf.train.RMSPropOptimizer.decay = 0.95
tf.train.RMSPropOptimizer.momentum = 0.0
tf.train.RMSPropOptimizer.epsilon = 0.00001
tf.train.RMSPropOptimizer.centered = True

# atari_lib.create_atari_environment.game_name = 'Pong'
# Deterministic ALE version used in the DQN Nature paper (Mnih et al., 2015).
# atari_lib.create_atari_environment.sticky_actions = False
# create_agent.agent_name = 'dqn'
Runner.num_iterations = 200
Runner.training_steps = 250000  # agent steps
Runner.evaluation_steps = 125000  # agent steps
Runner.max_steps_per_episode = 27000  # agent steps

AtariPreprocessing.terminal_on_life_loss = True

WrappedReplayBuffer.replay_capacity = 1000000
WrappedReplayBuffer.batch_size = 32
'''

# parse this config
gin.parse_config(gin_config, skip_unknown=False)


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
# Train DQN on Cartpole
#dqn_runner = create_runner(DQN_PATH, schedule='continuous_train')
#print('\n\n\nStart Training...\n\n\n')
#dqn_runner.run_experiment()
#print('\n\n\nDone training\n\n\n')
#dqn4 (5/28) - reward := -0.01 + 1*K - 3*D + log(S,100) + 5*L
#dqn5 (6/02) - same reward, but wall_offset = 0
#dqn7 (6/04) - final reward
DQN_PATH = '/tmp/bubble_dqn7'

# import main run()
from dopamine.discrete_domains import run_experiment

# config main file
gin_files = []
# bindings.....
gin_bindings = ['Runner.evaluation_steps=0']

# # code from train.main()
# tf.logging.set_verbosity(tf.logging.INFO)
# run_experiment.load_gin_configs(gin_files, gin_bindings)
# runner = run_experiment.create_runner(DQN_PATH)

# # start run
# runner.run_experiment()


## Thread for updating status

In [6]:
# Thread for update canvas
import threading, time
def get_ioloop():
    import IPython, zmq
    ipython = IPython.get_ipython()
    if ipython and hasattr(ipython, 'kernel'):
        return zmq.eventloop.ioloop.IOLoop.instance()
# The IOloop is shared
ioloop = get_ioloop()
# Main Thread
class MyThread(threading.Thread):
    '''Thread for drawing into canvas in live'''
    def __init__(self, sleep = 0.5, name = 'my'):
        super().__init__()
        self._quit = threading.Event()
        self.sleep = 0.5
        self.name = name
        self.start()    
    def run(self):
        while not self._quit.isSet():
            def update_progress():
                if self._quit.isSet():
                    return
                self.display()
            time.sleep(self.sleep)
            ioloop.add_callback(update_progress)
        print("! T[{}].Quit()".format(self.name))
    def quit(self):
        self._quit.set()
    def display(self):
        pass

# display basic 
from ipycanvas import Canvas
canvas = Canvas(width=640, height=480)
if canvas:
    canvas.stroke_text('hello canvas! -------------', 0, 10)
# show canvas in here.
canvas

Canvas(height=480, width=640)

In [7]:
# Helper for Canvas
#canvas.fill_style = 'green'
#canvas.fill_rect(25, 25, 100, 100)
#canvas.clear_rect(45, 45, 60, 60)
def drawPlot2Canvas(fig = None, x=0, y=0):
    '''draw current plt to canvas at (x,y)'''
    fig = plt.gcf() if fig is None else fig
    plt.close()          # not to update on screen.
    fig.canvas.draw()    # draw fig to canvas
    arr = np.array(fig.canvas.renderer._renderer)
    print('! arr = {}'.format(np.shape(arr)))
    h, w, d = np.shape(arr)
    print('! w,h,d = {}'.format(w))
    cv = Canvas(width=w, height=h)
    cv.put_image_data(arr, 0, 0)
    cv.stroke_rect(x, y, x+w-1, y+h-1)
    canvas.clear_rect(x,y,x+w,y+h)
    canvas.draw_image(cv, x, y)
def drawText2Canvas(txt='msg!', x=10, y=10):
    w,h,o = 200,10,10
    #canvas.fill_style = 'green'
    #canvas.fill_rect(x, y-o, x+w, y+h-o)
    canvas.clear_rect(x, y-o, x+w, y+h-o)
    canvas.stroke_text(txt, x, y)
# draw plot....
fig = plt.figure(1)
plt.plot([[1,3],[3,3],[7,1]])
# draw plot-to-canvas
drawPlot2Canvas(fig, x=0)
drawText2Canvas('hello world')

! arr = (288, 432, 4)
! w,h,d = 432


In [ ]:
#drawText2Canvas('......................')

### support Multi-Processing

In [8]:
from multiprocessing import Process, Queue
# process list
proc_list = []
proc_queue = None

# train function
def processTrain(name = 'train', Q = None):
    global gin_files, gin_bindings, DQN_PATH
    from dopamine.discrete_domains import run_experiment
    Q.put('init!') if Q else None
    tf.logging.set_verbosity(tf.logging.INFO)
    run_experiment.load_gin_configs(gin_files, gin_bindings)
    runner = run_experiment.create_runner(DQN_PATH)
    # access to env
    env = runner._environment
    o = env.reset()
    Q.put('! o({}) = {}'.format(type(o), o[0:10,0,]))
    Q.put('start!') if Q else None
    runner.run_experiment()
    Q.put('! P[{}].stop()'.format(name))

# train thread
def startProcessTrain(target = None):
    global proc_queue, proc_list
    target = target if target is not None else processTrain
    proc_queue = Queue() if proc_queue is None else proc_queue
    proc = Process(target = target, args = ('T0', proc_queue))
    proc_list.append(proc)
    proc.start()
    return proc

# stop(or kill) processes
def stopProcess():
    global proc_list
    for proc in proc_list:
        proc.terminate()
        proc.join()

In [ ]:
# trainer = startProcessTrain()

In [ ]:
# stop
# stopProcess()
# show process
# !ps -ax | grep python
# proc_queue

### MyTrainer and MyThread

In [9]:
from dopamine.discrete_domains import run_experiment
# MyRunner for Train
# - report every episode status.
class MyRunner(run_experiment.Runner):
    def __init__(self, base_dir, create_agent_fn):
        '''initialize runner'''
        super(MyRunner, self).__init__(base_dir, create_agent_fn)
        self._load_logger()
    def _run_one_episode(self):
        '''override to post episode status'''
        global proc_queue
        episode_length, episode_return = super(MyRunner, self)._run_one_episode()
        data = {'episode':{'length': episode_length, 'return': episode_return }}
        #proc_queue.put('! epsode[len,ret] = {},{}'.format(episode_length, episode_return))
        proc_queue.put(data)
        return episode_length, episode_return
    def _load_logger(self):
        '''load logger to save into file'''
        import logging, os
        # get TF logger
        log = logging.getLogger('tensorflow')
        log.setLevel(logging.DEBUG)        
        # create file handler which logs even debug messages
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        fh = logging.FileHandler(os.path.join(DQN_PATH, 'tensorflow.log'))
        fh.setLevel(logging.INFO)
        fh.setFormatter(formatter)
        log.addHandler(fh)


#! start runner
def startMyRunner(name = 'train', Q = None):
    global gin_files, gin_bindings, DQN_PATH
    from dopamine.discrete_domains import run_experiment
    Q.put('! start: my-runner') if Q else None
    tf.logging.set_verbosity(tf.logging.INFO)
    run_experiment.load_gin_configs(gin_files, gin_bindings)
    runner = MyRunner(DQN_PATH, run_experiment.create_agent)
    runner.run_experiment()
    Q.put('! P[{}].stop()'.format(name)) if Q else None
#! start process of runner
startProcessTrain(target = startMyRunner)

<Process(Process-1, started)>

! create-retro-game: BubbleBobble-Nes/Level01
! RetroPreprocessing: wall_offset=0, step_penality=-0.001
INFO:tensorflow:Creating DQNAgent agent with the following parameters:
INFO:tensorflow:	 gamma: 0.990000
INFO:tensorflow:	 update_horizon: 1.000000
INFO:tensorflow:	 min_replay_history: 50000
INFO:tensorflow:	 update_period: 4
INFO:tensorflow:	 target_update_period: 10000
INFO:tensorflow:	 epsilon_train: 0.100000
INFO:tensorflow:	 epsilon_eval: 0.050000
INFO:tensorflow:	 epsilon_decay_period: 1000000
INFO:tensorflow:	 tf_device: /gpu:1
INFO:tensorflow:	 use_staging: True
INFO:tensorflow:	 optimizer: <tensorflow.python.training.rmsprop.RMSPropOptimizer object at 0x7efb8cdcf0f0>
INFO:tensorflow:	 max_tf_checkpoints_to_keep: 4
INFO:tensorflow:Creating a OutOfGraphReplayBuffer replay memory with the following parameters:
INFO:tensorflow:	 observation_shape: (84, 84)
INFO:tensorflow:	 observation_dtype: <class 'numpy.uint8'>
INFO:tensorflow:	 terminal_dtype: <class 'numpy.uint8'>
INFO:ten

#### Train Results (01/Jun/2020) 
```pre
INFO:tensorflow:Starting iteration 87
INFO:tensorflow:Average undiscounted return per training episode: 19.29
INFO:tensorflow:Average training steps per second: 98.36
INFO:tensorflow:Starting iteration 88
INFO:tensorflow:Average undiscounted return per training episode: 17.34
INFO:tensorflow:Starting iteration 89
INFO:tensorflow:Average undiscounted return per training episode: 18.19
INFO:tensorflow:Starting iteration 90
INFO:tensorflow:Average undiscounted return per training episode: 16.46
INFO:tensorflow:Starting iteration 91
INFO:tensorflow:Average undiscounted return per training episode: 18.53
INFO:tensorflow:Starting iteration 92
INFO:tensorflow:Average undiscounted return per training episode: 18.22
INFO:tensorflow:Starting iteration 99
INFO:tensorflow:Average undiscounted return per training episode: 17.893
INFO:tensorflow:Starting iteration 100
INFO:tensorflow:Average undiscounted return per training episode: 18.24
INFO:tensorflow:Starting iteration 101
INFO:tensorflow:Average undiscounted return per training episode: 19.01
INFO:tensorflow:Starting iteration 102
INFO:tensorflow:Average undiscounted return per training episode: 19.94
INFO:tensorflow:Starting iteration 103
INFO:tensorflow:Average undiscounted return per training episode: 17.44
INFO:tensorflow:Starting iteration 104
INFO:tensorflow:Average undiscounted return per training episode: 17.876
INFO:tensorflow:Starting iteration 105
INFO:tensorflow:Average undiscounted return per training episode: 17.42
INFO:tensorflow:Starting iteration 106
INFO:tensorflow:Average undiscounted return per training episode: 17.595
INFO:tensorflow:Starting iteration 107
INFO:tensorflow:Average undiscounted return per training episode: 17.779
```

In [12]:
# MyThread for status display
class MyTrainStatus(MyThread):
    def __init__(self):
        super().__init__(name='status')
        self.episodes = np.array([[0,0]])
        print('! MyTrainStatus({})'.format(self.name))
    def display(self):
        global canvas, proc_queue, plt
        episodes = []
        # pop all queue...
        while not proc_queue.empty():
            msg = proc_queue.get()
            if msg and 'episode' in msg:
                E = msg['episode']
                episodes.append([E['length'], E['return']])
        # print('>> episodes = {}'.format(episodes))
        # draw plot if len > 0
        if len(episodes) > 0:
            arr = np.array(episodes)
            print('>> arr = {}'.format(arr))
            # draw plot...
            if 1>0:
                self.episodes = np.vstack((self.episodes, arr))
                #print('>> self.episodes = {}'.format(self.episodes))            
                #fig = plt.figure(1)
                #plt.plot(self.episodes)
                fig, ax1 = plt.subplots()
                ax2 = ax1.twinx()
                ax1.plot(self.episodes[:,0], 'g-')
                ax2.plot(self.episodes[:,1], 'b-')
                ax1.set_xlabel('episode count')
                ax1.set_ylabel('length', color='g')
                ax2.set_ylabel('return', color='b')
                drawPlot2Canvas(fig)
                
#! start thread for status
tstatus = MyTrainStatus()

! MyTrainStatus(status)


In [ ]:
episode_length, episode_return = 1,3
msg = {'episode':{'length': episode_length, 'return': episode_return }}
proc_queue.put(msg)
print('> msg.org = {}'.format(msg))

In [ ]:
# stop - thread of status
tstatus.quit() if tstatus else None

In [13]:
# stop - process of train
stopProcess()

KeyboardInterrupt: 